In [1]:
import dynamicrouting_summary as dr
import pandas as pd
import oursin as urchin
import numpy as np
import pyarrow.parquet as pq 
import npc_lims
import npc_session

In [7]:
VERSION = '0.0.224'

In [24]:
connection = npc_lims.get_probe_target_db()

cursor = connection.execute(
    f"SELECT * FROM channel_ccf_coords"
)
channel_ccf_coords = cursor.fetchall()
df_ccf = pd.DataFrame(channel_ccf_coords)
df_ccf.columns

Index(['index', 'session', 'MID', 'Day', 'Probe', 'Implant', 'Hole', 'Rig',
       'Channel_annotation_file', 'Channel_0_AP',
       ...
       'Channel_381_ML', 'Channel_381_region', 'Channel_382_AP',
       'Channel_382_DV', 'Channel_382_ML', 'Channel_382_region',
       'Channel_383_AP', 'Channel_383_DV', 'Channel_383_ML',
       'Channel_383_region'],
      dtype='object', length=1545)

In [25]:
cursor = connection.execute(
    f"SELECT * FROM min_distance_to_region"
)
min_distance = cursor.fetchall()
df_min_distance = pd.DataFrame(min_distance)
areas_of_interest = df_min_distance.columns[8:].to_list()
df_min_distance

,index,session,MID,Day,Probe,Implant,Hole,Rig,ACAd,ACAv,...,VISrl,VL,VM,VPL,VPM,VTA,ZI,AUDp,CLA,MGd
0,0,1167400342,599657,1,A,41,None,NP1,894.241234,293.496541,...,3717.397571,1511.714669,1757.501399,2187.950086,2065.528254,2957.023173,2435.497622,4777.061628,3258.406881,None
1,1,1167400342,599657,1,B,41,None,NP1,3766.656459,3092.767608,...,2537.813094,2216.655435,1915.572059,1958.401341,1584.688959,1707.117166,1822.079975,3884.160363,4155.860785,None
2,2,1167400342,599657,1,C,41,None,NP1,4842.923394,4219.463946,...,1073.084355,2332.296716,2315.841831,1433.220425,1365.191267,2270.079219,1508.884649,1979.643105,3250.989101,None
3,3,1167400342,599657,1,D,41,None,NP1,4770.699363,4167.564594,...,758.046498,2302.547298,1846.667289,1370.515003,1223.321302,1499.822519,1050.299251,1458.094812,3412.316956,None
4,4,1167400342,599657,1,E,41,None,NP1,1620.032438,1035.375340,...,2304.300264,999.356398,1974.711467,2173.203355,2076.896767,3216.244474,2615.137481,3631.178804,3050.176347,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1411,1411,708016_2024-05-02,708016,4,B,2006,B2,NP3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
1412,1412,708016_2024-05-02,708016,4,C,2006,C1,NP3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
1413,1413,708016_2024-05-02,708016,4,D,2006,E2,NP3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
1414,1414,708016_2024-05-02,708016,4,E,2006,F1,NP3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [2]:
electrode_paths = npc_lims.get_all_cache_paths('electrodes')
units_paths = npc_lims.get_all_cache_paths('units')
electrode_groups_paths = npc_lims.get_all_cache_paths('electrode_groups')

In [5]:
electrodes_original = pd.concat(pd.read_parquet(electrode_path) for electrode_path in electrode_paths)
#units_structures = pd.concat(pd.read_parquet(unit_path) for unit_path in units_paths)[['structure', 'location', 'peak_channel', 'subject_id', 'session_idx', 'date', 
  #                                    'ccf_ap', 'ccf_dv', 'ccf_ml', 'group_name', 'default_qc', 'unit_id']]
electrode_groups = pd.concat(pd.read_parquet(electrode_group_path) for electrode_group_path in electrode_groups_paths)

In [8]:
electrodes_original = dr.add_bool_columns(electrodes_original, version=VERSION, session_ids=tuple(electrodes_original['session_id'].unique().tolist()))
#units_structures_with_bools = dr.add_bool_columns(units_structures, version=VERSION, session_ids=tuple(units_structures['session_id'].unique().tolist()))
#electrodes_original = electrodes_original[electrodes_original['subject_id'].isin(subjects_passing_behavior)]

#units_structures_with_bools.dropna(inplace=True)
#units_structures_with_bools.columns

In [9]:
electrodes_original.columns

Index(['location', 'group_name', 'structure', 'x', 'y', 'z', 'channel',
       'rel_x', 'rel_y', 'reference', 'imp', 'raw_location', 'raw_structure',
       'session_idx', 'date', 'subject_id', 'session_id', 'is_ephys',
       'is_templeton', 'is_training', 'is_dynamic_routing', 'is_opto'],
      dtype='object')

In [10]:
import ipywidgets

electrodes = electrodes_original.dropna()
electrodes = electrodes[~(electrodes['session_id'].str.contains('662892'))]
electrodes = electrodes[~(electrodes['structure'].str.islower())]
electrodes = electrodes[electrodes['is_dynamic_routing'] == True]
electrodes.columns

Index(['location', 'group_name', 'structure', 'x', 'y', 'z', 'channel',
       'rel_x', 'rel_y', 'reference', 'imp', 'raw_location', 'raw_structure',
       'session_idx', 'date', 'subject_id', 'session_id', 'is_ephys',
       'is_templeton', 'is_training', 'is_dynamic_routing', 'is_opto'],
      dtype='object')

In [11]:
electrodes = electrodes.merge(electrode_groups, left_on=['session_id', 'group_name'], right_on=['session_id', 'name'])
electrodes.columns

Index(['location_x', 'group_name', 'structure', 'x', 'y', 'z', 'channel',
       'rel_x', 'rel_y', 'reference', 'imp', 'raw_location', 'raw_structure',
       'session_idx_x', 'date_x', 'subject_id_x', 'session_id', 'is_ephys',
       'is_templeton', 'is_training', 'is_dynamic_routing', 'is_opto',
       'description', 'location_y', 'name', 'session_idx_y', 'date_y',
       'subject_id_y'],
      dtype='object')

In [12]:
from ipywidgets import interact, Box

In [13]:
def get_subject_colors(electrodes_structures_area: pd.DataFrame) -> list[tuple[int, int, int]]: 
    subject_colors = dr.utils.generate_subject_random_colors(electrodes_structures_area)
    subject_color_list = []

    for index, row in electrodes_structures_area.iterrows():
        if 'MID' in electrodes_structures_area.columns:
            subject_color_list.append(subject_colors[str(row['MID'])])
        else:
            subject_color_list.append(subject_colors[str(row['subject_id_x'])])
    
    return subject_color_list

In [26]:
unique_implants = df_ccf['Implant'].unique().tolist()
unique_implants.append('All')
implants_dropdown = ipywidgets.Dropdown(options=unique_implants, value=unique_implants[-1], description='Implants', disabled=False)

unique_holes = df_ccf['Hole'].unique().tolist()
unique_holes.append('All')
holes_dropdown = ipywidgets.Dropdown(options=unique_holes, value=unique_holes[-1], description='Holes', disabled=False)

unique_probes = df_ccf['Probe'].unique().tolist()
unique_probes.append('All')
probes_dropdown = ipywidgets.Dropdown(options=unique_probes, value=unique_probes[-1], description='Probes', disabled=False)

In [27]:
def update_3d_viewer_based_on_trajectory() -> tuple[list]:
    urchin.ccf25.clear()
    urchin.clear_meshes()

    if implants_dropdown.value == 'All':
        pmeshes = urchin.meshes.create(len(electrodes)) #creates 2 primitives, stored in list pmeshes
        electrodes_sessions = electrodes
        coords = electrodes[['x', 'z', 'y']].to_numpy().tolist()
    else:
        probe_target_sessions = df_ccf[(df_ccf['Implant'] == implants_dropdown.value) & (df_ccf['Hole'] == holes_dropdown.value)
                                    & (df_ccf['Probe'] == probes_dropdown.value)]
        probe_target_sessions_filtered = []
        """
        for session in probe_target_sessions:
            try:
                session_record = npc_session.SessionRecord(session)
                probe_target_sessions_filtered.append(f"{session_record.subject}_{session_record.date}_{session_record.idx}")
            except ValueError:
                pass
        """
        #electrodes_sessions = probe_target_sessions
        #lectrodes_sessions = electrodes_sessions[electrodes_sessions['Probe'] == probes_dropdown.value]
        
        coords = probe_target_sessions.iloc[:, 9:].to_numpy()
        
        ccf_coords = None
        for coord in coords:
            coord_reshape = coord.reshape((384, 4))
            if ccf_coords is None:
                ccf_coords = coord_reshape
            else:
                ccf_coords = np.concatenate((ccf_coords, coord_reshape))

    print(f"Implant {implants_dropdown.value} Hole {holes_dropdown.value} Probe {probes_dropdown.value}")
    print(f"Mice Day: {probe_target_sessions[['MID', 'Day']].values.tolist()}")
    print(f"Areas: {np.unique(ccf_coords[:, 3]).tolist()}")
    urchin.ccf25.root.set_visibility(True)
    urchin.ccf25.root.set_material('transparent-lit')
    urchin.ccf25.root.set_alpha(0.15)
    urchin.ccf25.root.set_color("#000000")

    # reorder to AP/ML/DV for Urchin and make a list of lists
    #ccf_coords[:, [2, 1]] = ccf_coords[:, [1, 2]]
    #ccf_coords = ccf_coords[ccf_coords[:, 0] > 0]
    coords_list = []
    for ccf_coord in ccf_coords:
        if ccf_coord[0] > 0 and not np.isnan(ccf_coord[0]):
            coords_list.append([ccf_coord[0] * 25, ccf_coord[2] * 25, ccf_coord[1] * 25])
    
    pmeshes = urchin.meshes.create(len(coords_list)) #creates 2 primitives, stored in list pmeshes
    callback_ids = [mesh.id for mesh in pmeshes]
    session_list = probe_target_sessions[['session', 'MID','Hole', 'Implant', 'Probe']].to_numpy().tolist()
    sizes_list = [[0.03,0.03,0.03]]*len(coords_list)
    urchin.meshes.set_positions(pmeshes,coords_list) #sets the positions of the primitives
    urchin.meshes.set_colors(pmeshes, ['#FF0000'] * len(coords_list))
    urchin.meshes.set_scales(pmeshes, sizes_list)

    area_list = urchin.ccf25.get_areas(np.unique(ccf_coords[:, 3]).tolist())
    
    urchin.ccf25.set_visibilities(area_list, True, urchin.utils.Side.LEFT)
    urchin.ccf25.set_materials(area_list, 'transparent-unlit', "left")
    urchin.ccf25.set_alphas(area_list, 0.2, "left")
    

    return callback_ids, session_list

In [28]:
def filter_by_implant(implant: str):
    if implant == 'All':
        holes_dropdown.options = unique_holes
        holes_dropdown.value = 'All'
    else:
        df_implant = df_ccf[df_ccf['Implant'] == implant]
        holes_dropdown.options = df_implant['Hole'].unique().tolist()
        holes_dropdown.value = holes_dropdown.options[0]

In [29]:
def filter_by_implant_hole(implant: str, hole: str):
    if implant == 'All' and hole == 'All':
        probes_dropdown.options = unique_probes
        probes_dropdown.value = 'All'
    else:
        df_implant_hole = df_ccf[(df_ccf['Implant'] == implant) & (df_ccf['Hole'] == hole)]
        probes_dropdown.options = sorted(df_implant_hole['Probe'].unique().tolist())
        probes_dropdown.value = probes_dropdown.options[0]

In [30]:
urchin.setup()

(urchin) Client is already connected. Use ID: e0cecaae


In [31]:
urchin.ccf25.load()

(Warning) Atlas was already loaded, the renderer can have issues if you try to load an atlas twice.


In [32]:
@interact(x=['coronal', 'axial', 'sagittal'])
def camera_rotation(x):
    urchin.camera.main.set_rotation(x)
    urchin.camera.main.set_mode('perspective')
    urchin.camera.main.set_background_color('#ffffff')

interactive(children=(Dropdown(description='x', options=('coronal', 'axial', 'sagittal'), value='coronal'), Ou…

In [33]:
@interact(x=(5, 60))
def zoom(x=45):
    urchin.camera.main.set_zoom(60-x)

interactive(children=(IntSlider(value=45, description='x', max=60, min=5), Output()), _dom_classes=('widget-in…

In [34]:
items = [implants_dropdown, holes_dropdown, probes_dropdown]
box = Box(children=items)
box

Box(children=(Dropdown(description='Implants', index=9, options=('41', 'football', 'TS1', 'TS5', '2002', 'temp…

In [35]:
filter_by_implant(implants_dropdown.value)

In [90]:
filter_by_implant_hole(implants_dropdown.value, holes_dropdown.value)

In [113]:
call_back_ids, session_list = update_3d_viewer_based_on_trajectory()

Implant 2002 Hole A1 Probe A
Mice Day: [[632296, 1], [632296, 3], [632295, 1], [632295, 3], [636740, 1], [636740, 3], [637483, 1], [637483, 3], [638387, 1], [638387, 3], [640887, 2], [640887, 4], [637488, 2], [637488, 4], [644867, 1], [644867, 2], [649943, 1], [649943, 2], [642504, 2], [666986, 3], [666986, 4], [667252, 1], [667252, 2], [681532, 2]]
Areas: ['ACAd5', 'ACAd6a', 'ACAd6b', 'CA1', 'CA2', 'CA3', 'CL', 'DG-mo', 'DG-po', 'DG-sg', 'Eth', 'FC', 'HPF', 'IMD', 'LD', 'LH', 'LP', 'MB', 'MD', 'MH', 'MOp1', 'MOp2/3', 'MOp5', 'MOp6a', 'MOp6b', 'MOs1', 'MOs2/3', 'MOs5', 'MOs6a', 'MOs6b', 'OP', 'PAG', 'PO', 'PRC', 'PVT', 'RSPagl1', 'RSPagl2/3', 'RSPagl5', 'RSPagl6a', 'RSPagl6b', 'RSPd1', 'RSPd2/3', 'RSPd5', 'RSPd6a', 'RSPd6b', 'RSPv5', 'RSPv6a', 'RSPv6b', 'SPA', 'SSp-ll1', 'SSp-ll2/3', 'SSp-ll4', 'SSp-ll5', 'SSp-ll6a', 'SSp-ll6b', 'SSp-tr1', 'SSp-tr2/3', 'SSp-tr4', 'SSp-tr5', 'SSp-tr6a', 'SSp-tr6b', 'TH', 'Track not annotated', 'V3', 'VL', 'VPMpc', 'alv', 'ccb', 'ccs', 'chpl', 'cing', 'd

In [36]:
unique_areas = electrodes['structure'].unique()
unique_areas.sort()
area_dropdown = ipywidgets.Dropdown(options=unique_areas, value=unique_areas[0], description='CCF Structure Area', disabled=False)

def update_3d_viewer(area: str) -> tuple[list]:
    urchin.ccf25.clear()
    urchin.clear_meshes()
    units_structures_with_bools_dr_area = electrodes[electrodes['structure'] == area]
    electrodes_structures_area = electrodes[electrodes['structure'] == area]

    print(f"{area}: {len(electrodes_structures_area['subject_id_x'].unique())} mice")
    print(f"{area}: {len(electrodes_structures_area['session_id'].unique())} sessions")
    #print(f"{area}: {len(units_structures_with_bools_dr_area)} units")

    pmeshes = urchin.meshes.create(len(electrodes_structures_area)) #creates 2 primitives, stored in list pmeshes
    coords = electrodes_structures_area[['x', 'z', 'y']].to_numpy().tolist()
    callback_ids = [mesh.id for mesh in pmeshes]
    session_list = electrodes_structures_area[['session_id', 'location_y', 'group_name']].to_numpy().tolist()

    urchin.ccf25.root.set_visibility(True)
    urchin.ccf25.root.set_material('transparent-lit')
    urchin.ccf25.root.set_alpha(0.15)
    urchin.ccf25.root.set_color("#000000")

    # reorder to AP/ML/DV for Urchin and make a list of lists
    coords_list = [[x[0], x[1], x[2]] for x in coords]
    print(session_list)
    sizes_list = [[0.03,0.03,0.03]]*len(coords)

    urchin.meshes.set_positions(pmeshes,coords_list) #sets the positions of the primitives
    urchin.meshes.set_colors(pmeshes, get_subject_colors(units_structures_with_bools_dr_area))
    urchin.meshes.set_scales(pmeshes, sizes_list)

    area_list = urchin.ccf25.get_areas([area])

    urchin.ccf25.set_visibilities(area_list, True, urchin.utils.Side.LEFT)
    urchin.ccf25.set_materials(area_list, 'transparent-unlit', "left")
    urchin.ccf25.set_alphas(area_list, 0.2, "left")

    return callback_ids, session_list

area_dropdown


Dropdown(description='CCF Structure Area', options=('ACAd', 'ACAv', 'ACB', 'AD', 'AId', 'AIv', 'AMd', 'AMv', '…

In [37]:
call_back_ids, session_list = update_3d_viewer(area_dropdown.value)

VM: 4 mice
VM: 4 sessions
[['660023_2023-08-09_0', '2002 E1', 'probeE'], ['660023_2023-08-09_0', '2002 E1', 'probeE'], ['660023_2023-08-09_0', '2002 E1', 'probeE'], ['660023_2023-08-09_0', '2002 E1', 'probeE'], ['660023_2023-08-09_0', '2002 E1', 'probeE'], ['660023_2023-08-09_0', '2002 E1', 'probeE'], ['660023_2023-08-09_0', '2002 E1', 'probeE'], ['660023_2023-08-09_0', '2002 E1', 'probeE'], ['660023_2023-08-09_0', '2002 E1', 'probeE'], ['660023_2023-08-09_0', '2002 E1', 'probeE'], ['668755_2023-08-29_0', '2002 E3', 'probeE'], ['668755_2023-08-29_0', '2002 E3', 'probeE'], ['668755_2023-08-29_0', '2002 E3', 'probeE'], ['668755_2023-08-29_0', '2002 E3', 'probeE'], ['668755_2023-08-29_0', '2002 E3', 'probeE'], ['668755_2023-08-29_0', '2002 E3', 'probeE'], ['668755_2023-08-29_0', '2002 E3', 'probeE'], ['668755_2023-08-29_0', '2002 E3', 'probeE'], ['668755_2023-08-29_0', '2002 E3', 'probeE'], ['668755_2023-08-29_0', '2002 E3', 'probeE'], ['668755_2023-08-29_0', '2002 E3', 'probeE'], ['66875

In [ ]:
area_dropdown_min_distance = ipywidgets.Dropdown(options=areas_of_interest, value=areas_of_interest[0], description='Areas for min distance viewer', disabled=False)
area_dropdown_min_distance

In [ ]:
@interact(min_distance_to_area=(250, 500))
def min_distance_viewer(min_distance_to_area=300):
    urchin.ccf25.clear()
    urchin.clear_meshes()

    probe_target_sessions = df_min_distance[(df_min_distance[area_dropdown_min_distance.value] / 25) <= min_distance_to_area]
    probe_target_sessions_filtered = []
    for session in probe_target_sessions['session'].tolist():
        try:
            session_record = npc_session.SessionRecord(session)
            probe_target_sessions_filtered.append(f"{session_record.subject}_{session_record.date}_{session_record.idx}")
        except ValueError:
            pass
    
    electrodes_sessions = electrodes[electrodes['session_id'].isin(probe_target_sessions_filtered)]
    electrodes_sessions = electrodes_sessions[electrodes_sessions['structure'] == area_dropdown_min_distance.value]
    pmeshes = urchin.meshes.create(len(electrodes_sessions)) #creates 2 primitives, stored in list pmeshes
    coords = electrodes_sessions[['x', 'z', 'y']].to_numpy().tolist()
   
    print(f"Sessions: {electrodes_sessions['session_id'].unique().tolist()}")
  
    
    urchin.ccf25.root.set_visibility(True)
    urchin.ccf25.root.set_material('transparent-lit')
    urchin.ccf25.root.set_alpha(0.15)
    urchin.ccf25.root.set_color("#000000")

    # reorder to AP/ML/DV for Urchin and make a list of lists
    coords_list = [[x[0], x[1], x[2]] for x in coords]
    session_list = electrodes_sessions['session_id'].tolist()
    sizes_list = [[0.07,0.07,0.07]]*len(coords)

    urchin.meshes.set_positions(pmeshes,coords_list) #sets the positions of the primitives
    urchin.meshes.set_colors(pmeshes, get_subject_colors(electrodes_sessions))
    urchin.meshes.set_scales(pmeshes, sizes_list)

    area_list = urchin.ccf25.get_areas(electrodes_sessions['structure'].unique().tolist())
    urchin.ccf25.set_visibilities(area_list, True, urchin.utils.Side.LEFT)
    urchin.ccf25.set_materials(area_list, 'transparent-unlit', "left")
    urchin.ccf25.set_alphas(area_list, 0.2, "left")
    

In [ ]:
counts = {'area': [], 'num_units': [], 'num_sessions': [], 'num_subjects': []}
units_structure_areas_no_white_matter = units_structures_with_bools_dr[~(units_structures_with_bools_dr['structure'].str.islower())]
for area in units_structure_areas_no_white_matter['structure'].unique():
    units_structure_areas = units_structure_areas_no_white_matter[units_structure_areas_no_white_matter['structure'] == area]
    counts['area'].append(area)
    counts['num_sessions'].append(len(units_structure_areas['session_id'].unique()))
    counts['num_subjects'].append(len(units_structure_areas['subject_id_x'].unique()))
    counts['num_units'].append(len(units_structure_areas))

df_counts = pd.DataFrame(counts)

In [ ]:
df_counts.style.applymap(lambda x: 'background-color : red' if x>=3 else '', subset=['num_subjects']).applymap(lambda x: 'background-color : red' if x>150 else '', subset=['num_units'])

In [45]:
import zarr
times = zarr.open(npc_lims.get_cache_path('spike_times', version='any'))

In [46]:
from IPython.display import display, Image, clear_output
import matplotlib.pyplot as plt
%matplotlib inline

def display_session_trajectory(index: str):
    session = session_list[call_back_ids.index(index)]
    session_id = npc_session.SessionRecord(session[0])
    hole_implant = session[1]
    probe = session[2]
    clicked_unit_id = session[3]
    clear_output()
    display(session)
    unit_ids_session_probe_hole = units_structures_with_bools_dr[(units_structures_with_bools_dr['session_id'] == session_id)
                                    & (units_structures_with_bools_dr['location_y'] == hole_implant)
                                    & (units_structures_with_bools_dr['structure'] == area_dropdown.value)
                                    & (units_structures_with_bools_dr['group_name'] == probe)]['unit_id'].tolist()
    #display(unit_ids_session_probe_hole)
    fig = dr.plot_unit_by_id(clicked_unit_id, times[f'/{session_id.subject}_{session_id.date}/{clicked_unit_id}'][:])
    display(fig)
    plt.close(fig)
    unit_ids_session_probe_hole.remove(clicked_unit_id)
    for unit_id in unit_ids_session_probe_hole[0:5]:
        fig = dr.plot_unit_by_id(unit_id, times[f'/{session_id.subject}_{session_id.date}/{unit_id}'][:])
        display(fig)
        plt.close(fig)
    
urchin.meshes.callback = display_session_trajectory